In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../../LIB/')
from env import ENV
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [2]:
X = pd.read_csv(ENV.bureau_balance_ori.value)

In [3]:
class ordinal_encoder:
    def __init__(self,fillna='NAN_FILL'):
        self.fillna = fillna
    
    def fit(self,sr,NA_VALUE=None,realNA2Nega1=True):
        """
        spycify the value which has already been encoded. make it to -1 after encoding
        """
        self.NA_VALUE=NA_VALUE
        order = sr.fillna(self.fillna).value_counts()
        if self.NA_VALUE is not None:
            order[NA_VALUE] = 9999999

        if realNA2Nega1:
            order[self.fillna] = 99999999
        order = order.sort_values(ascending=True)


        self.mapping = pd.Series(index=order.index.values, data=list(range(1,len(order)+1)))
        if self.NA_VALUE is not None:
            self.mapping[NA_VALUE] = -1

        if realNA2Nega1:
            self.mapping[self.fillna] = -1
    
    def transform(self,sr):
        return sr.fillna(self.fillna).map(self.mapping)
        

In [4]:
def calculate_na(ser):
    return np.sum(ser.isnull())

def view_hist(ser):
    plt.hist(ser, range=(min(ser.values), max(ser.values)))
    
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [5]:
p = scan_nan_portion(X)
p = p.sort_values()
print(p.describe())
oe = ordinal_encoder()

count    3.0
mean     0.0
std      0.0
min      0.0
25%      0.0
50%      0.0
75%      0.0
max      0.0
dtype: float64


# Export NAN

In [6]:
p.iloc[0:20]

SK_ID_BUREAU      0.0
MONTHS_BALANCE    0.0
STATUS            0.0
dtype: float64

# Data Clean

## 0 - 20

In [7]:
category_20 = []
failed = []
for each in p.iloc[:20].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_20.append(each)
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('========= End ==================')
print(failed)
# category_20.remove()

count    2.729992e+07
mean     6.036297e+06
std      4.923489e+05
min      5.001709e+06
25%      5.730933e+06
50%      6.070821e+06
75%      6.431951e+06
max      6.842888e+06
Name: SK_ID_BUREAU, dtype: float64
----
int64
value counts: 817395
NA percentage: 0.0
========= End ==================
count    2.729992e+07
mean    -3.074169e+01
std      2.386451e+01
min     -9.600000e+01
25%     -4.600000e+01
50%     -2.500000e+01
75%     -1.100000e+01
max      0.000000e+00
Name: MONTHS_BALANCE, dtype: float64
----
int64
value counts: 97
NA percentage: 0.0
========= End ==================
count     27299925
unique           8
top              C
freq      13646993
Name: STATUS, dtype: object
----
object
STATUS converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
value counts: 8
NA percentage: 0.0
========= End ==================
['STATUS']


In [13]:
col = 'STATUS'
X[col].value_counts()

9    13646993
7     7499507
6     5810482
5      242347
4       62406
3       23419
2        8924
1        5847
Name: STATUS, dtype: int64

### Ordinal Encoding

In [12]:
col = 'STATUS'

oe.fit(X[col],NA_VALUE='XNA')
X[col] = oe.transform(X[col])
category_20.append(col)


# Saving

In [14]:
X.to_pickle(ENV.bureau_balance_clean.value)

In [15]:
np.sum(X.isnull())

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [16]:
categori_col = list(set(category_20))
categori_col

['STATUS']

In [17]:
import pickle
categori_col_exist = pickle.load(open(ENV.clean_categorical_col.value,'rb'))
categori_col = list(set(categori_col + categori_col_exist))
pickle.dump(categori_col,open(ENV.clean_categorical_col.value,'wb'))